
<div class="alert alert-block alert-info"><font size="5"><center><b><u> Sprint 11_Practicant amb training i test sets
   </u></b></center></font>
\begin{align*}Marc\:Breysse\end{align*}

<span style=color:blue>*Familiaritza't amb la programació científica mitjantçant la llibreria SKLearn / Scikitlearn.*</span>

# <span style=color:darkblue>Nivell 1</span>

<u style=color:blue>Exercici 1:</u>

<span style=color:blue>Parteix el conjunt de dades DelayedFlights.csv en train i test. Estudia els dos conjunts per separat, a nivell descriptiu</span>

In [97]:
import os
import pandas as pd
import numpy as np
import datetime 

import seaborn as sns

from sklearn.model_selection import train_test_split
    
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [98]:
#Build Year/Month/day columns to Date format while loading dataset</span>
DF = pd.read_csv('DelayedFlights.csv',
                 parse_dates= {"Date" : ["Year","Month","DayofMonth"]})
DF.shape

(1936758, 28)

In [99]:
DF.head()

,Date,Unnamed: 0,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008-01-03,0,4,2003.0,1955,2211.0,2225,WN,335,N712SW,...,4.0,8.0,0,N,0,NaN,NaN,NaN,NaN,NaN
1,2008-01-03,1,4,754.0,735,1002.0,1000,WN,3231,N772SW,...,5.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
2,2008-01-03,2,4,628.0,620,804.0,750,WN,448,N428WN,...,3.0,17.0,0,N,0,NaN,NaN,NaN,NaN,NaN
3,2008-01-03,4,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,...,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0
4,2008-01-03,5,4,1940.0,1915,2121.0,2110,WN,378,N726SW,...,4.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN


In [100]:
DF.columns

Index(['Date', 'Unnamed: 0', 'DayOfWeek', 'DepTime', 'CRSDepTime', 'ArrTime',
       'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'],
      dtype='object')

In [101]:
DF.nunique()

Date                     366
Unnamed: 0           1936758
DayOfWeek                  7
DepTime                 1438
CRSDepTime              1207
ArrTime                 1440
CRSArrTime              1364
UniqueCarrier             20
FlightNum               7499
TailNum                 5366
ActualElapsedTime        673
CRSElapsedTime           515
AirTime                  650
ArrDelay                1128
DepDelay                1058
Origin                   303
Dest                     304
Distance                1419
TaxiIn                   180
TaxiOut                  332
Cancelled                  2
CancellationCode           4
Diverted                   2
CarrierDelay             983
WeatherDelay             599
NASDelay                 574
SecurityDelay            156
LateAircraftDelay        564
dtype: int64

<span style=color:green><b> 1. DATASET PREPROCESSING - CLEANING

<span style=color:green>1.1 Convert 'DepTime' and others into datetime and then minutes</span>

In [102]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1936758 entries, 0 to 1936757
Data columns (total 28 columns):
 #   Column             Dtype         
---  ------             -----         
 0   Date               datetime64[ns]
 1   Unnamed: 0         int64         
 2   DayOfWeek          int64         
 3   DepTime            float64       
 4   CRSDepTime         int64         
 5   ArrTime            float64       
 6   CRSArrTime         int64         
 7   UniqueCarrier      object        
 8   FlightNum          int64         
 9   TailNum            object        
 10  ActualElapsedTime  float64       
 11  CRSElapsedTime     float64       
 12  AirTime            float64       
 13  ArrDelay           float64       
 14  DepDelay           float64       
 15  Origin             object        
 16  Dest               object        
 17  Distance           int64         
 18  TaxiIn             float64       
 19  TaxiOut            float64       
 20  Cancelled          int64

In [103]:
# converting input time value to datetime.
def conv_time(time_val):
    if pd.isnull(time_val):
        return np.nan
    else:
            # replace 24:00 o'clock with 00:00 o'clock:
        if time_val == 2400: time_val = 0
            # creating a 4 digit value out of input value:
        time_val = "{0:04d}".format(int(time_val))
            # creating a time datatype out of input value: 
        time_formatted = datetime.time(int(time_val[0:2]), int(time_val[2:4]))
    return time_formatted

In [104]:
DF['DepTime'] = DF['DepTime'].apply(conv_time)
DF['CRSDepTime'] = DF['CRSDepTime'].apply(conv_time)
DF['ArrTime'] = DF['ArrTime'].apply(conv_time)
DF['CRSArrTime'] = DF['CRSArrTime'].apply(conv_time)

In [105]:
DF.head()

,Date,Unnamed: 0,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008-01-03,0,4,20:03:00,19:55:00,22:11:00,22:25:00,WN,335,N712SW,...,4.0,8.0,0,N,0,NaN,NaN,NaN,NaN,NaN
1,2008-01-03,1,4,07:54:00,07:35:00,10:02:00,10:00:00,WN,3231,N772SW,...,5.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
2,2008-01-03,2,4,06:28:00,06:20:00,08:04:00,07:50:00,WN,448,N428WN,...,3.0,17.0,0,N,0,NaN,NaN,NaN,NaN,NaN
3,2008-01-03,4,4,18:29:00,17:55:00,19:59:00,19:25:00,WN,3920,N464WN,...,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0
4,2008-01-03,5,4,19:40:00,19:15:00,21:21:00,21:10:00,WN,378,N726SW,...,4.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN


In [106]:
#Convert time (HH:MM:SS) to minutes
DF['DepTime'] = pd.to_datetime(DF['DepTime'], format = '%H:%M:%S')
DF['DepTime'] = DF['DepTime'].dt.hour * 60 + DF['DepTime'].dt.minute
DF['CRSDepTime'] = pd.to_datetime(DF['CRSDepTime'], format = '%H:%M:%S')
DF['CRSDepTime'] =DF['CRSDepTime'].dt.hour * 60 + DF['CRSDepTime'].dt.minute
DF['ArrTime'] = pd.to_datetime(DF['ArrTime'], format = '%H:%M:%S')
DF['ArrTime'] = DF['ArrTime'].dt.hour * 60 + DF['ArrTime'].dt.minute
DF['CRSArrTime'] = pd.to_datetime(DF['CRSArrTime'], format = '%H:%M:%S')
DF['CRSArrTime'] = DF['CRSArrTime'].dt.hour * 60 + DF['CRSArrTime'].dt.minute

In [107]:
DF.head()

,Date,Unnamed: 0,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008-01-03,0,4,1203,1195,1331.0,1345,WN,335,N712SW,...,4.0,8.0,0,N,0,NaN,NaN,NaN,NaN,NaN
1,2008-01-03,1,4,474,455,602.0,600,WN,3231,N772SW,...,5.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
2,2008-01-03,2,4,388,380,484.0,470,WN,448,N428WN,...,3.0,17.0,0,N,0,NaN,NaN,NaN,NaN,NaN
3,2008-01-03,4,4,1109,1075,1199.0,1165,WN,3920,N464WN,...,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0
4,2008-01-03,5,4,1180,1155,1281.0,1270,WN,378,N726SW,...,4.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN


In [108]:
DF.describe().round(1)

,Unnamed: 0,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,...,Distance,TaxiIn,TaxiOut,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,1936758.0,1936758.0,1936758.0,1936758.0,1929648.0,1936758.0,1936758.0,1928371.0,1936560.0,1928371.0,...,1936758.0,1929648.0,1936303.0,1936758.0,1936758.0,1247488.0,1247488.0,1247488.0,1247488.0,1247488.0
mean,3341651.2,4.0,922.6,891.5,976.8,992.0,2184.3,133.3,134.3,108.3,...,765.7,6.8,18.2,0.0,0.0,19.2,3.7,15.0,0.1,25.3
std,2066065.0,2.0,270.5,255.2,330.1,279.1,1944.7,72.1,71.3,68.6,...,574.5,5.3,14.3,0.0,0.1,43.5,21.5,33.8,2.0,42.1
min,0.0,1.0,0.0,0.0,0.0,0.0,1.0,14.0,-25.0,0.0,...,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,1517452.5,2.0,723.0,695.0,795.0,805.0,610.0,80.0,82.0,58.0,...,338.0,4.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,3242558.0,4.0,945.0,910.0,1035.0,1025.0,1543.0,116.0,116.0,90.0,...,606.0,6.0,14.0,0.0,0.0,2.0,0.0,2.0,0.0,8.0
75%,4972466.8,6.0,1140.0,1095.0,1230.0,1214.0,3422.0,165.0,165.0,137.0,...,998.0,8.0,21.0,0.0,0.0,21.0,0.0,15.0,0.0,33.0
max,7009727.0,7.0,1439.0,1439.0,1439.0,1439.0,9742.0,1114.0,660.0,1091.0,...,4962.0,240.0,422.0,1.0,1.0,2436.0,1352.0,1357.0,392.0,1316.0


<span style=color:green>1.2 Delete columns which are not relevant for 'ArrDelay study' in further sprints</span>

In [109]:
item_counts = DF["Cancelled"].value_counts()
item_counts

0    1936125
1        633
Name: Cancelled, dtype: int64

In [110]:
item_counts = DF["CancellationCode"].value_counts()
item_counts

N    1936125
B        307
A        246
C         80
Name: CancellationCode, dtype: int64

In [111]:
item_counts = DF["Diverted"].value_counts()
item_counts

0    1929004
1       7754
Name: Diverted, dtype: int64

<span style=color:green>*The 3 above columns have no information about 'ArrDelay' and can be removed.*</span>

<span style=color:green>*Also 'FlightNum', 'TailNum' and delay types will be removed*</span>

<span style=color:green>*Finally since DEPARTURE_DELAY = SCHEDULED_DEPARTURE - DEPARTURE_TIME and the same for ARRIVAL, we can remove 'DepTime', 'ArrTime','ActualElapsedTime' and 'CRSElapsedTime'*</span>

In [112]:
DF = DF[['DayOfWeek', 'CRSDepTime',
       'CRSArrTime','ActualElapsedTime', 'CRSElapsedTime','ArrDelay',
       'DepDelay','TaxiIn', 'TaxiOut','UniqueCarrier','Origin', 'Dest','Date']]

In [113]:
DF.head()

,DayOfWeek,CRSDepTime,CRSArrTime,ActualElapsedTime,CRSElapsedTime,ArrDelay,DepDelay,TaxiIn,TaxiOut,UniqueCarrier,Origin,Dest,Date
0,4,1195,1345,128.0,150.0,-14.0,8.0,4.0,8.0,WN,IAD,TPA,2008-01-03
1,4,455,600,128.0,145.0,2.0,19.0,5.0,10.0,WN,IAD,TPA,2008-01-03
2,4,380,470,96.0,90.0,14.0,8.0,3.0,17.0,WN,IND,BWI,2008-01-03
3,4,1075,1165,90.0,90.0,34.0,34.0,3.0,10.0,WN,IND,BWI,2008-01-03
4,4,1155,1270,101.0,115.0,11.0,25.0,4.0,10.0,WN,IND,JAX,2008-01-03


<span style=color:green>1.3 Null Values</span>

In [114]:
#check percentage of missing values
missing_DF = DF.isnull().sum(axis=0).reset_index()
missing_DF.columns = ['variable', 'missing values']
missing_DF['filling factor (%)']=(DF.shape[0]-missing_DF['missing values'])/DF.shape[0]*100
missing_DF.sort_values('filling factor (%)').reset_index(drop = True)

,variable,missing values,filling factor (%)
0,ActualElapsedTime,8387,99.566957
1,ArrDelay,8387,99.566957
2,TaxiIn,7110,99.632892
3,TaxiOut,455,99.976507
4,CRSElapsedTime,198,99.989777
5,DayOfWeek,0,100.000000
6,CRSDepTime,0,100.000000
7,CRSArrTime,0,100.000000
8,DepDelay,0,100.000000
9,UniqueCarrier,0,100.000000


In [115]:
# Eliminate rows with null value since less than 0.5% of Null values
DF.dropna(inplace = True)
DF.shape

(1928371, 13)

<span style=color:green>1.4 Save dataset 'DelayedFlightsCleaned' for next sprints</span>

In [116]:
DF.to_csv(r'C:/Users/mio/Documents/IT ACADEMY/11_Practicant amb training i test sets/DelayedFlightsCleaned.csv')

<span style=color:green><b> 2. TRAIN-TEST SPLIT

In [135]:
DFC = pd.read_csv('DelayedFlightsCleaned.csv')

In [136]:
from sklearn.model_selection import train_test_split

X, y = DFC.drop('ArrDelay',axis=1), DFC.ArrDelay
print(X.shape, y.shape)

(1928371, 13) (1928371,)


In [137]:
# split into train test sets (80% Train - 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1542696, 13) (385675, 13) (1542696,) (385675,)


In [138]:
y_train.describe().round(3)

count    1542696.000
mean          42.181
std           56.726
min         -109.000
25%            9.000
50%           24.000
75%           56.000
max         1707.000
Name: ArrDelay, dtype: float64

In [139]:
y_test.describe().round(3)

count    385675.000
mean         42.274
std          57.021
min         -59.000
25%           9.000
50%          24.000
75%          56.000
max        2461.000
Name: ArrDelay, dtype: float64

In [140]:
# difference betweem 'ArrDelay' mean of y_test and y-train < 0.5%
((y_train.mean()-y_test.mean())/y_test.mean())*100

-0.21772828180862272

In [141]:
# split into train test sets (99% Train - 1% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1909087, 13) (19284, 13) (1909087,) (19284,)


In [142]:
y_train.describe().round(3)

count    1909087.000
mean          42.192
std           56.767
min         -109.000
25%            9.000
50%           24.000
75%           56.000
max         2461.000
Name: ArrDelay, dtype: float64

In [143]:
y_test.describe().round(3)

count    19284.000
mean        43.007
std         58.554
min        -51.000
25%          9.000
50%         24.000
75%         57.000
max       1116.000
Name: ArrDelay, dtype: float64

In [126]:
# difference betweem 'ArrDelay' mean of y_test and y-train > 1%
((y_train.mean()-y_test.mean())/y_test.mean())*100

-1.8956649997875836

# <span style=color:darkblue>Nivell 2</span>

<u style=color:blue>Exercici 2:</u>

<span style=color:blue>Aplica algun procés de transformació (estandarditzar les dades numèriques, crear columnes dummies, polinomis...).<span>

<span style=color:green>2.1 Create dummy variables with 'Unique Carrier' variable*</span>

In [128]:
# Create dummy variables
dummy_UniqueCarrier_type = pd.get_dummies(X_train['UniqueCarrier'], prefix='UniqueCarrier', drop_first=True)

# Drop unnecessary columns
to_drop = ['Unnamed: 0','UniqueCarrier']
X_train.drop(to_drop, axis=1, inplace=True)

# Merge into a single data frame
X_train = pd.concat([X_train, dummy_UniqueCarrier_type], axis=1)
X_train.head()

KeyError: 'UniqueCarrier'

In [91]:
# Apply dummy to X_test
dummy_UniqueCarrier_type = pd.get_dummies(X_test['UniqueCarrier'], prefix='UniqueCarrier', drop_first=True)
to_drop = ['Unnamed: 0','UniqueCarrier']
X_test.drop(to_drop, axis=1, inplace=True)
# Merge into a single data frame
X_test = pd.concat([X_test, dummy_UniqueCarrier_type], axis=1)
X_test.head()

,DayOfWeek,CRSDepTime,CRSArrTime,ActualElapsedTime,CRSElapsedTime,DepDelay,TaxiIn,TaxiOut,Origin,Dest,...,UniqueCarrier_HA,UniqueCarrier_MQ,UniqueCarrier_NW,UniqueCarrier_OH,UniqueCarrier_OO,UniqueCarrier_UA,UniqueCarrier_US,UniqueCarrier_WN,UniqueCarrier_XE,UniqueCarrier_YV
942808,6,642,734,96.0,92.0,39.0,6.0,17.0,TUS,LAX,...,0,0,0,0,1,0,0,0,0,0
1484347,1,1115,1185,62.0,70.0,9.0,6.0,6.0,MAF,DFW,...,0,1,0,0,0,0,0,0,0,0
1566787,6,975,1037,112.0,122.0,9.0,7.0,14.0,CLT,ORD,...,0,0,0,0,0,0,1,0,0,0
1256237,2,1040,1160,149.0,120.0,28.0,5.0,59.0,ATL,DCA,...,0,0,0,0,0,0,0,0,0,0
651063,5,1000,1037,92.0,97.0,13.0,7.0,8.0,CVG,MEM,...,0,0,0,0,0,0,0,0,0,0


In [94]:
from sklearn.preprocessing import StandardScaler

# Scale selected columns by index
# Set up scaler
scaler = StandardScaler().fit(X_train.iloc[:,:8])
X_train.iloc[:,:8] = scaler.transform(X_train.iloc[:,:8])
X_train.head()
#  Scale X_Test data
X_test.iloc[:,:8] = scaler.transform(X_test.iloc[:,:8])

# <span style=color:darkblue>Nivell 3</span>

<u style=color:blue>Exercici 3:</u>

<span style=color:blue>Resumeix les noves columnes generades de manera estadística i gràfica.<span>


In [95]:
X_train.describe().round(3)

,DayOfWeek,CRSDepTime,CRSArrTime,ActualElapsedTime,CRSElapsedTime,DepDelay,TaxiIn,TaxiOut,UniqueCarrier_AA,UniqueCarrier_AQ,...,UniqueCarrier_HA,UniqueCarrier_MQ,UniqueCarrier_NW,UniqueCarrier_OH,UniqueCarrier_OO,UniqueCarrier_UA,UniqueCarrier_US,UniqueCarrier_WN,UniqueCarrier_XE,UniqueCarrier_YV
count,1909087.000,1909087.000,1909087.000,1909087.000,1909087.000,1909087.000,1909087.000,1909087.000,1909087.000,1909087.00,...,1909087.000,1909087.000,1909087.000,1909087.000,1909087.000,1909087.000,1909087.000,1909087.000,1909087.000,1909087.000
mean,0.000,-0.000,-0.000,-0.000,-0.000,-0.000,0.000,-0.000,0.099,0.00,...,0.004,0.073,0.041,0.027,0.068,0.073,0.051,0.195,0.053,0.035
std,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.299,0.02,...,0.062,0.261,0.198,0.163,0.252,0.260,0.220,0.396,0.225,0.183
min,-1.496,-3.494,-3.556,-1.656,-2.179,-0.696,-1.293,-1.273,0.000,0.00,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,-0.995,-0.771,-0.671,-0.740,-0.733,-0.584,-0.534,-0.574,0.000,0.00,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
50%,0.007,0.072,0.118,-0.240,-0.255,-0.358,-0.154,-0.295,0.000,0.00,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
75%,1.009,0.797,0.795,0.440,0.433,0.186,0.226,0.195,0.000,0.00,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
max,1.510,2.145,1.602,13.610,7.382,45.520,44.267,28.224,1.000,1.00,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000


In [96]:
X_test.describe().round(3)

,DayOfWeek,CRSDepTime,CRSArrTime,ActualElapsedTime,CRSElapsedTime,DepDelay,TaxiIn,TaxiOut,UniqueCarrier_AA,UniqueCarrier_AQ,...,UniqueCarrier_HA,UniqueCarrier_MQ,UniqueCarrier_NW,UniqueCarrier_OH,UniqueCarrier_OO,UniqueCarrier_UA,UniqueCarrier_US,UniqueCarrier_WN,UniqueCarrier_XE,UniqueCarrier_YV
count,19284.000,19284.000,19284.000,19284.000,19284.000,19284.000,19284.000,19284.000,19284.000,19284.00,...,19284.000,19284.000,19284.000,19284.000,19284.000,19284.000,19284.000,19284.000,19284.000,19284.000
mean,-0.006,0.003,-0.007,0.009,0.009,0.014,-0.004,0.009,0.099,0.00,...,0.004,0.071,0.044,0.028,0.068,0.075,0.051,0.194,0.053,0.033
std,1.000,1.002,1.010,1.003,1.006,1.031,1.004,1.013,0.299,0.02,...,0.064,0.256,0.204,0.165,0.251,0.264,0.220,0.395,0.225,0.180
min,-1.496,-3.455,-3.552,-1.531,-1.547,-0.696,-1.293,-1.273,0.000,0.00,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,-0.995,-0.771,-0.685,-0.740,-0.733,-0.584,-0.534,-0.574,0.000,0.00,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
50%,0.007,0.072,0.118,-0.226,-0.241,-0.358,-0.154,-0.295,0.000,0.00,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
75%,1.009,0.808,0.799,0.454,0.433,0.205,0.226,0.195,0.000,0.00,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
max,1.510,2.145,1.602,6.754,7.031,20.487,32.497,14.803,1.000,1.00,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
